<a href="https://colab.research.google.com/github/HongJaeKwon/machine-learning/blob/master/HJK_PG_REINFORCE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/gdrive', force_remount=True)
# /gdrive/My Drive/ (폴더명)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
# 그림파일로 렌더링 하도록 패키지 설정
!apt-get install python-opengl -y
!apt install xvfb -y
!pip install pyvirtualdisplay
!pip install piglet

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  libgle3
The following NEW packages will be installed:
  python-opengl
0 upgraded, 1 newly installed, 0 to remove and 31 not upgraded.
Need to get 496 kB of archives.
After this operation, 5,416 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python-opengl all 3.1.0+dfsg-1 [496 kB]
Fetched 496 kB in 2s (237 kB/s)
Selecting previously unselected package python-opengl.
(Reading database ... 144433 files and directories currently installed.)
Preparing to unpack .../python-opengl_3.1.0+dfsg-1_all.deb ...
Unpacking python-opengl (3.1.0+dfsg-1) ...
Setting up python-opengl (3.1.0+dfsg-1) ...
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  xvfb
0 upgraded, 1 newly installed, 0 to remove and 31 not upgraded.
Need to get 784 kB of 

In [0]:
# 필요한 모듈 설치
import tensorflow as tf
import gym
from IPython import display
import cv2
from pyvirtualdisplay import Display
from IPython import display
import matplotlib.pyplot as plt
from collections import deque
import numpy as np
import random
%matplotlib inline
Display().start()

xdpyinfo was not found, X start can not be checked! Please install xdpyinfo!


<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1024x768x24', ':1001'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1024x768x24', ':1001'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [0]:
# 카트폴 게임 환경을 만듦
env = gym.make("CartPole-v1")

In [0]:
env.render('rgb_array')

In [0]:
# 2 - 액션 종류 슈 (아웃풋)
action_num=env.action_space.n
# 4 - 상태 종류 수 (인풋)
state_num=env.observation_space.shape[0]

In [0]:
# pg 모델 만들기 - REINFORCE 알고리즘(가장 기초)
pg_model=tf.keras.models.Sequential()
pg_model.add(tf.keras.layers.Dense(256,input_shape=(state_num,),activation='relu'))
pg_model.add(tf.keras.layers.Dense(action_num,activation='softmax'))
opt=tf.keras.optimizers.Adam(0.0005)

In [0]:
# 에피소드 수만큼 학습
episode_count=1000

# 점수를 기록할 리스트
scores = []
memory=[]

# 디스카운트 팩터 정의
discount_rate=0.99

for episode in range(episode_count):
    state = env.reset()
    # 차원을 맞추어 준다
    state = np.reshape(state, [1, state_num])
    done = False
    total_reward = 0
    while not done:
        p=pg_model.predict(state)[0]
        action=np.random.choice(range(action_num),p=p)
        next_state, reward, done, _ = env.step(action)
        next_state = np.reshape(next_state, [1, state_num])
        i=(state,action,reward/100.,next_state,done)
        # 메모리에 작업 내용을 기록한다
        memory.append(i)
        # 다음상태를 현사태로 변경하여 계속 진행한다
        state = next_state
        total_reward += reward

    # 에피소드가 끝나면 PG 학습 시작
    G=0
    for s,a,r,n_s,d in memory[::-1]:
        G=r+discount_rate * G
        variable = pg_model.trainable_variables
        with tf.GradientTape() as tape:
            p=pg_model(s)[0][a]
            loss = -tf.math.log(p) * G
        grad=tape.gradient(loss,variable)
        opt.apply_gradients(zip(grad,variable))   
    
    memory=[]
    scores.append(total_reward)
    if(total_reward>450):
        pg_model.save('/gdrive/My Drive/hjk_pg_reinforce_model.h5')
    mean_score = np.mean(scores)
    
    print(episode+1,total_reward)

    if (episode+1) % 20 == 0:
        print("Episode %d: Mean survival = %0.2lf in %d episodes" %(episode+1, mean_score, 20))
        if mean_score >= 400:
            break
        scores = []

env.close() 

1 11.0
2 11.0
3 16.0
4 27.0
5 29.0
6 52.0
7 19.0
8 35.0
9 11.0
10 13.0
11 16.0
12 36.0
13 14.0
14 11.0
15 15.0
16 29.0
17 27.0
18 51.0
19 27.0
20 24.0
Episode 20: Mean survival = 23.70 in 20 episodes
21 15.0
22 26.0
23 21.0
24 18.0
25 13.0
26 13.0
27 26.0
28 22.0
29 33.0
30 45.0
31 17.0
32 24.0
33 44.0
34 20.0
35 19.0
36 38.0
37 34.0
38 25.0
39 28.0
40 38.0
Episode 40: Mean survival = 25.95 in 20 episodes
41 27.0
42 15.0
43 18.0
44 40.0
45 26.0
46 19.0
47 18.0
48 19.0
49 31.0
50 34.0
51 17.0
52 36.0
53 18.0
54 16.0
55 17.0
56 59.0
57 86.0
58 43.0
59 49.0
60 38.0
Episode 60: Mean survival = 31.30 in 20 episodes
61 72.0
62 62.0
63 26.0
64 34.0
65 40.0
66 55.0
67 18.0
68 33.0
69 43.0
70 24.0
71 37.0
72 44.0
73 100.0
74 46.0
75 65.0
76 32.0
77 81.0
78 82.0
79 32.0
80 71.0
Episode 80: Mean survival = 49.85 in 20 episodes
81 34.0
82 80.0
83 88.0
84 119.0
85 41.0
86 70.0
87 43.0
88 97.0
89 42.0
90 42.0
91 88.0
92 46.0
93 92.0
94 114.0
95 151.0
96 169.0
97 110.0
98 179.0
99 291.0
100 189.0
Epi

In [0]:
pg_model=tf.keras.models.load_model('/gdrive/My Drive/hjk_pg_reinforce_model.h5')

In [0]:
env = gym.make('CartPole-v1')
state=env.reset()
state = np.reshape(state, [1, state_num])
done=False
# img = plt.imshow(env.render('rgb_array')) # only call this once
total_reward=0
img_avi=np.zeros((400,600,3))
fcc=cv2.VideoWriter_fourcc(*'DIVX')
out=cv2.VideoWriter('/gdrive/My Drive/hjk_pg_reinforce.avi',fcc,10.0,(600,400))
while not done:
    # img.set_data(env.render('rgb_array')) # just update the data
    # display.display(plt.gcf())
    # display.clear_output(wait=True)
    img_avi=env.render('rgb_array')
    action = np.argmax(pg_model.predict(state)[0])
    # action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    next_state = np.reshape(next_state, [1, state_num])
    state = next_state
    total_reward += reward
    out.write(np.uint8(img_avi))
print(total_reward)
out.release()
cv2.destroyAllWindows()

500.0
